[ref1](https://github.com/L1aoXingyu/pytorch-beginner/blob/master/08-AutoEncoder/Variational_autoencoder.py)
[ref2](https://github.com/pytorch/examples/blob/master/vae/main.py)

In [27]:
import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if not os.path.exists('./vae_img'):
    os.mkdir('./vae_img')


def to_img(x):
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor()
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = MNIST('../../data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE().to(device)

def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD


optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_idx, (img, _) in enumerate(dataloader):
        img = img.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(img)
        loss = loss_function(recon_batch, img, mu, logvar)
        loss.backward()
        train_loss += loss.data.item()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(img),
                len(dataloader.dataset), 100. * batch_idx / len(dataloader),
                loss.data.item() / len(img)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(dataloader.dataset)))
    if epoch % 10 == 0:
        save = to_img(recon_batch.cpu().data)
        save_image(save, './vae_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './vae.pth')

Train Epoch: 0 [0/60000 (0%)]	Loss: 548.700806
Train Epoch: 0 [12800/60000 (21%)]	Loss: 187.096481
Train Epoch: 0 [25600/60000 (43%)]	Loss: 156.791885
Train Epoch: 0 [38400/60000 (64%)]	Loss: 136.749588
Train Epoch: 0 [51200/60000 (85%)]	Loss: 131.278442
====> Epoch: 0 Average loss: 164.9630
Train Epoch: 1 [0/60000 (0%)]	Loss: 127.938126
Train Epoch: 1 [12800/60000 (21%)]	Loss: 122.163132
Train Epoch: 1 [25600/60000 (43%)]	Loss: 121.013260
Train Epoch: 1 [38400/60000 (64%)]	Loss: 114.547012
Train Epoch: 1 [51200/60000 (85%)]	Loss: 114.103790
====> Epoch: 1 Average loss: 121.5355
Train Epoch: 2 [0/60000 (0%)]	Loss: 114.780197
Train Epoch: 2 [12800/60000 (21%)]	Loss: 116.092239
Train Epoch: 2 [25600/60000 (43%)]	Loss: 111.983383
Train Epoch: 2 [38400/60000 (64%)]	Loss: 113.149506
Train Epoch: 2 [51200/60000 (85%)]	Loss: 109.113365
====> Epoch: 2 Average loss: 114.5389
Train Epoch: 3 [0/60000 (0%)]	Loss: 111.213425
Train Epoch: 3 [12800/60000 (21%)]	Loss: 114.537674
Train Epoch: 3 [25600/

Train Epoch: 27 [51200/60000 (85%)]	Loss: 103.878998
====> Epoch: 27 Average loss: 102.9715
Train Epoch: 28 [0/60000 (0%)]	Loss: 102.905487
Train Epoch: 28 [12800/60000 (21%)]	Loss: 101.422470
Train Epoch: 28 [25600/60000 (43%)]	Loss: 99.431427
Train Epoch: 28 [38400/60000 (64%)]	Loss: 101.512962
Train Epoch: 28 [51200/60000 (85%)]	Loss: 104.940544
====> Epoch: 28 Average loss: 102.9075
Train Epoch: 29 [0/60000 (0%)]	Loss: 100.712761
Train Epoch: 29 [12800/60000 (21%)]	Loss: 99.849503
Train Epoch: 29 [25600/60000 (43%)]	Loss: 104.279411
Train Epoch: 29 [38400/60000 (64%)]	Loss: 97.729355
Train Epoch: 29 [51200/60000 (85%)]	Loss: 99.926208
====> Epoch: 29 Average loss: 102.7751
Train Epoch: 30 [0/60000 (0%)]	Loss: 101.010033
Train Epoch: 30 [12800/60000 (21%)]	Loss: 106.193062
Train Epoch: 30 [25600/60000 (43%)]	Loss: 101.738518
Train Epoch: 30 [38400/60000 (64%)]	Loss: 102.713417
Train Epoch: 30 [51200/60000 (85%)]	Loss: 102.138870
====> Epoch: 30 Average loss: 102.7350
Train Epoch: 31

Train Epoch: 55 [25600/60000 (43%)]	Loss: 102.072159
Train Epoch: 55 [38400/60000 (64%)]	Loss: 98.807472
Train Epoch: 55 [51200/60000 (85%)]	Loss: 104.056854
====> Epoch: 55 Average loss: 101.4293
Train Epoch: 56 [0/60000 (0%)]	Loss: 100.768448
Train Epoch: 56 [12800/60000 (21%)]	Loss: 99.587395
Train Epoch: 56 [25600/60000 (43%)]	Loss: 99.282021
Train Epoch: 56 [38400/60000 (64%)]	Loss: 104.144135
Train Epoch: 56 [51200/60000 (85%)]	Loss: 101.462448
====> Epoch: 56 Average loss: 101.3241
Train Epoch: 57 [0/60000 (0%)]	Loss: 102.744904
Train Epoch: 57 [12800/60000 (21%)]	Loss: 100.403900
Train Epoch: 57 [25600/60000 (43%)]	Loss: 104.365715
Train Epoch: 57 [38400/60000 (64%)]	Loss: 100.268646
Train Epoch: 57 [51200/60000 (85%)]	Loss: 107.638008
====> Epoch: 57 Average loss: 101.3094
Train Epoch: 58 [0/60000 (0%)]	Loss: 99.151543
Train Epoch: 58 [12800/60000 (21%)]	Loss: 101.669373
Train Epoch: 58 [25600/60000 (43%)]	Loss: 98.667969
Train Epoch: 58 [38400/60000 (64%)]	Loss: 100.996666
Tr

Train Epoch: 83 [12800/60000 (21%)]	Loss: 101.890587
Train Epoch: 83 [25600/60000 (43%)]	Loss: 100.443893
Train Epoch: 83 [38400/60000 (64%)]	Loss: 100.610580
Train Epoch: 83 [51200/60000 (85%)]	Loss: 97.958260
====> Epoch: 83 Average loss: 100.5869
Train Epoch: 84 [0/60000 (0%)]	Loss: 98.864983
Train Epoch: 84 [12800/60000 (21%)]	Loss: 105.226692
Train Epoch: 84 [25600/60000 (43%)]	Loss: 102.424026
Train Epoch: 84 [38400/60000 (64%)]	Loss: 99.398926
Train Epoch: 84 [51200/60000 (85%)]	Loss: 100.609871
====> Epoch: 84 Average loss: 100.5951
Train Epoch: 85 [0/60000 (0%)]	Loss: 98.535004
Train Epoch: 85 [12800/60000 (21%)]	Loss: 98.534485
Train Epoch: 85 [25600/60000 (43%)]	Loss: 99.649567
Train Epoch: 85 [38400/60000 (64%)]	Loss: 97.687988
Train Epoch: 85 [51200/60000 (85%)]	Loss: 101.776588
====> Epoch: 85 Average loss: 100.5822
Train Epoch: 86 [0/60000 (0%)]	Loss: 99.504158
Train Epoch: 86 [12800/60000 (21%)]	Loss: 101.434708
Train Epoch: 86 [25600/60000 (43%)]	Loss: 100.177391
Train